In [1]:
import sys
import os 
sys.path.append('..')  #adds the Root Directory to the system path
from BL_CalciumAnalysis.image_analysis_methods import ImageAnalysis

In [2]:
print(sys.executable) #print the path of the Python executable being used, which should point to the Python interpreter in your Conda environment.

/Users/cresp1el/anaconda3/envs/biolumi_calcium_imaging/bin/python


In [3]:
project_folder = '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis' #path to the folder containing the raw data to be analyzed (i.e. the folder containing the folders for each experiment)
analysis = ImageAnalysis(project_folder)

print(analysis.directory_df)
analysis.directory_df


                       directory_name  \
0      g21_12072023_estim_10hz_na_blk   
1      g22_12072023_estim_10hz_na_blk   
2      g23_12072023_estim_10hz_na_blk   
3    c33_12092023_estim_10hz_1xfz_blk   
4  c33_12092023_estim_10hz_1xfz_blk_1   
5    c32_12092023_estim_10hz_1xfz_blk   
6    c31_12092023_estim_10hz_1xfz_blk   

                                      directory_path  
0  /Volumes/MannySSD/cablam_imaging/raw_data_for_...  
1  /Volumes/MannySSD/cablam_imaging/raw_data_for_...  
2  /Volumes/MannySSD/cablam_imaging/raw_data_for_...  
3  /Volumes/MannySSD/cablam_imaging/raw_data_for_...  
4  /Volumes/MannySSD/cablam_imaging/raw_data_for_...  
5  /Volumes/MannySSD/cablam_imaging/raw_data_for_...  
6  /Volumes/MannySSD/cablam_imaging/raw_data_for_...  


,directory_name,directory_path
0,g21_12072023_estim_10hz_na_blk,/Volumes/MannySSD/cablam_imaging/raw_data_for_...
1,g22_12072023_estim_10hz_na_blk,/Volumes/MannySSD/cablam_imaging/raw_data_for_...
2,g23_12072023_estim_10hz_na_blk,/Volumes/MannySSD/cablam_imaging/raw_data_for_...
3,c33_12092023_estim_10hz_1xfz_blk,/Volumes/MannySSD/cablam_imaging/raw_data_for_...
4,c33_12092023_estim_10hz_1xfz_blk_1,/Volumes/MannySSD/cablam_imaging/raw_data_for_...
5,c32_12092023_estim_10hz_1xfz_blk,/Volumes/MannySSD/cablam_imaging/raw_data_for_...
6,c31_12092023_estim_10hz_1xfz_blk,/Volumes/MannySSD/cablam_imaging/raw_data_for_...


In [4]:
# Since the folders are directly inside the project_folder, you don't need to append any subdirectory name
data_files = analysis.list_files('') 
print(data_files) #print the list of files in the project folder 

directories = analysis.list_directories()
print(directories) #print the list of directories in the project folder 

['/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g21_12072023_estim_10hz_na_blk/g21_12072023_estim_10hz_na_blk_fluor_combined.tif', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g21_12072023_estim_10hz_na_blk/g21_12072023_estim_10hz_na_blk_fluor.csv', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g21_12072023_estim_10hz_na_blk/._g21_12072023_estim_10hz_na_blk_fluor.csv', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g22_12072023_estim_10hz_na_blk/g22_12072023_estim_10hz_na_blk_fluor_combined.tif', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g22_12072023_estim_10hz_na_blk/g22_12072023_estim_10hz_na_blk_fluor.csv', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g23_12072023_estim_10hz_na_blk/g23_12072023_estim_10hz_na_blk.tif', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/g23_12072023_estim_10hz_na_blk/g23_12072023_estim_10hz_na_blk_fluor.csv', '/Volumes/MannySSD/cablam_imaging/raw_data_for_analysis/c33_12092023_estim_